In [ ]:
# import pandas as pd
# import os

# ruta = 'data' 

# dataframes = []

# for archivo in os.listdir(ruta):
#     if archivo.endswith('.parquet'):
#         df = pd.read_parquet(os.path.join(ruta, archivo))
#         dataframes.append(df)

# df_concatenado = pd.concat(dataframes, ignore_index=True)
# df_concatenado.to_csv('resultado.csv', index=False)

In [ ]:
# pip3 install googletrans==3.1.0a0

In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

from googletrans import Translator
from transformers import Trainer, TrainingArguments, BertTokenizer, EncoderDecoderModel

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## **Traductor**

In [3]:
trans = Translator()
print(trans.translate('Pues hazme los deberes, furcia barata!').text)

Well, do my homework for me, you cheap whore!


## **Dataset**

In [4]:
csv = pd.read_csv('2_detox_dataset_trans.csv')
df = pd.DataFrame(csv)

train_dataframe, val_dataframe = train_test_split(df, train_size=0.9, random_state=42) # random_state=42 for reproducibility

train_toxic_texts = list(train_dataframe['toxic_sentence'])
train_neutral_texts = list(train_dataframe['neutral_sentence'])

val_toxic_texts = list(val_dataframe['toxic_sentence'])
val_neutral_texts = list(val_dataframe['neutral_sentence'])

## **Detoxificador**

In [5]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
toxic_encodings_train = tokenizer(train_toxic_texts, truncation=True, padding=True, return_tensors="pt")
neutral_encodings_train = tokenizer(train_neutral_texts, truncation=True, padding=True, return_tensors="pt")

toxic_encodings_val = tokenizer(val_toxic_texts, truncation=True, padding=True, return_tensors="pt")
neutral_encodings_val = tokenizer(val_neutral_texts, truncation=True, padding=True, return_tensors="pt")

# model = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-multilingual-cased", "bert-base-multilingual-cased")
model = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")

# Configura parámetros específicos para la generación de texto
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

# Hiperparámetros para la generación
model.config.min_length = 3
model.config.max_length = 64
model.config.no_repeat_ngram_size = 2
model.config.early_stopping = True

Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.layer.1.crossattention.self.key.bias', 'bert.e

In [6]:
class DetoxDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, target_encodings):
        self.encodings = encodings
        self.target_encodings = target_encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        # item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        # item['labels'] = torch.tensor(self.target_encodings['input_ids'][idx])
        item['labels'] = self.target_encodings['input_ids'][idx].clone().detach()
        return item

train_dataset = DetoxDataset(toxic_encodings_train, neutral_encodings_train)
val_dataset = DetoxDataset(toxic_encodings_val, neutral_encodings_val)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=2e-4,
    num_train_epochs=3,
    report_to=["none"], # para no pedir login de 'wandb' y otros
    fp16=True, # acelerar entrenaminento 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [7]:
trainer.train()

  0%|          | 0/306 [00:00<?, ?it/s]

c:\Users\aleja\anaconda3\envs\entornotorch\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:616: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
c:\Users\aleja\anaconda3\envs\entornotorch\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:636: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


  0%|          | 0/12 [00:00<?, ?it/s]

{'eval_loss': 1.5755783319473267, 'eval_runtime': 0.6631, 'eval_samples_per_second': 542.919, 'eval_steps_per_second': 18.097, 'epoch': 1.0}


  0%|          | 0/12 [00:00<?, ?it/s]

{'eval_loss': 1.5094608068466187, 'eval_runtime': 0.6104, 'eval_samples_per_second': 589.795, 'eval_steps_per_second': 19.66, 'epoch': 2.0}


  0%|          | 0/12 [00:00<?, ?it/s]

{'eval_loss': 1.451763391494751, 'eval_runtime': 0.6436, 'eval_samples_per_second': 559.326, 'eval_steps_per_second': 18.644, 'epoch': 3.0}
{'train_runtime': 77.709, 'train_samples_per_second': 125.082, 'train_steps_per_second': 3.938, 'train_loss': 1.4135100919436785, 'epoch': 3.0}


TrainOutput(global_step=306, training_loss=1.4135100919436785, metrics={'train_runtime': 77.709, 'train_samples_per_second': 125.082, 'train_steps_per_second': 3.938, 'train_loss': 1.4135100919436785, 'epoch': 3.0})

In [9]:
def detoxify_sentence(text: str):
    # Tokenizar la oración tóxica
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    # Mover los tensores a la GPU, si está disponible
    inputs = {key: val.to(device) for key, val in inputs.items()}
    
    # Generar la oración neutral
    outputs = model.generate(
        inputs['input_ids'],
        max_length=64,
        num_beams=4,
        # early_stopping=True
    )
    
    # Decodificar el resultado
    neutral_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return neutral_text

In [ ]:
test_sentences = [
    "Well, do my homework for me, you cheap whore!",
    "You are a stupid asshole."
]

for sentence in test_sentences:
    neutral_sentence = detoxify_sentence(sentence)
    # print(f"Original: {sentence} -> Neutral: {neutral_sentence}")
    print(f"Original: {sentence} -> Neutral: {neutral_sentence}") # ”

c:\Users\aleja\anaconda3\envs\entornotorch\lib\site-packages\transformers\generation\configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
c:\Users\aleja\anaconda3\envs\entornotorch\lib\site-packages\transformers\generation\configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
c:\Users\aleja\anaconda3\envs\entornotorch\lib\site-packages\transformers\generation\utils.py:1339: UserWarning: You have modified the pretrained model configuration to control generation. T

Original: Well, do my homework for me, you cheap whore! -> Neutral: i ',, “ ’ ”wisedneriladitionallices uncomfortable domestic journalists this heygrakel [unused285] ti [unused221] time [unused671] boeing [unused708] invite have [unused226]wee [unused683]
Original: You are a stupid asshole. -> Neutral: you are you you. you your you, you is.. your.,. are.?. is you? you “ ’ ”wise uncomfortable thisiladitionalgra inappropriate person are naive unpleasantdner useless


---

---

## **BUCLE DE TRADUCCIÓN** (26 mins)

In [ ]:
csv = pd.read_csv('detox_dataset.csv')
df = pd.DataFrame(csv)

trans = Translator()
# print(trans.translate(df.iloc[400]['toxic_sentence'], dest='en').text)
print(trans.translate("ታምራት ነገራ ፅንፈኞችን ከላይ እስከታች ሰብስበህ ሁለት ሰንበት ፀበልና ስልጠና ስጥልን", dest='en').text)

In [ ]:
csv_trans = pd.read_csv('detox_dataset_trans.csv')
df_trans = pd.DataFrame(csv_trans)

In [ ]:
for index in range(len(df)):
    traductor = Translator()
    toxic_translated = traductor.translate(df.iloc[index]['toxic_sentence'], dest='en').text
    neutral_translated = traductor.translate(df.iloc[index]['neutral_sentence'], dest='en').text
    new_row = [toxic_translated, neutral_translated]
    df_trans.loc[len(df_trans)] = new_row
    # break

df_trans.to_csv('detox_dataset_trans.csv', index=False)